In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [4]:
df=pd.read_csv("ds_test.csv")
df.insert(5,"FamSize",(df.Parch+df.SibSp),True)
df=df.drop(["Ticket","SibSp","Parch","Embarked","Fare"],axis=1)                             
#Add relatives together in a column to get the family size            
df["Age"].fillna(value=df["Age"].ffill(),inplace=True)
df

,PassengerId,Pclass,Name,Sex,Age,FamSize,Cabin
0,892,3,"Kelly, Mr. James",male,34.5,0,NaN
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,NaN
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,NaN
3,895,3,"Wirz, Mr. Albert",male,27.0,0,NaN
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,2,NaN
...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,28.0,0,NaN
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,C105
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,NaN
416,1308,3,"Ware, Mr. Frederick",male,38.5,0,NaN


In [5]:
#Age is missing calues, but we can fill it. Cabin is missing a lot, so we can not extrapolate based on existing data
df["Age"].fillna(value=df["Age"].mean(),inplace=True)

In [6]:
#Create a function to extract the deck code from the cabin number, X for NaN
def deck(cabin_no):
    deck_no="X"
    if not pd.isna(cabin_no):
        deck_no=re.search(r"([A-Za-z]+)",str(cabin_no)).group(0)
    return deck_no

In [7]:
#Put the results from the function in a new column
df["Deck"]=df["Cabin"].map(lambda x: deck(x))
df.drop(columns=["Cabin"],inplace=True)
#No missing values in the dataset

In [8]:
#Function to get name titles 
def title_gen(namen):
    title = re.search(r"(,[A-Za-z ]+\.?)", namen).group(0).replace(", ","")
    return title

df["Title"]=df["Name"].map(lambda x: title_gen(x))

In [9]:
#Function to extract the title from the name and sort them based on sex and social ranking
def title_mod(i):
    title=i["Title"]
    if title in ['Don.', 'Major.', 'Lady.', 'Sir.', 'Col.', 'Capt.',
       'the Countess.', 'Jonkheer.']:
        if i["Sex"]=="female":
            return "Noblewoman"
        else:
            return "Nobleman"
    elif title in ['Dr.','Mrs.', 'Miss.','Mme.','Ms.','Mlle.'] and i["Sex"]=="female":
        return "Mrs"
    else:
        return "Mr"

df["Title"]=df.apply(lambda x: title_mod(x), axis=1)
#We have created a column with each passenger's title, so we can drop the name column
df.drop(columns=["Name"],inplace=True)

In [10]:
answer=np.ones(418)
answer=answer.tolist()
for i in range(len(df)):
    if df.loc[i]["Age"]>=18:
        if df.loc[i]["Sex"]=="male":
            if df.loc[i]["Pclass"]==1:
                answer[i]*=0.35
            elif df.loc[i]["Pclass"]==2:
                answer[i]*=0.07
            else:
                answer[i]*=0.11
        else:
            if df.loc[i]["Pclass"]==1:
                answer[i]*=0.98
            elif df.loc[i]["Pclass"]==2:
                answer[i]*=0.90
            else:
                answer[i]*=0.48
    else:
        if df.loc[i]["Pclass"]==1:
            answer[i]*=0.8
        elif df.loc[i]["Pclass"]==2:
            answer[i]*=0.85
        else:
            answer[i]*=0.37

In [11]:
for i in range(len(df)):
    if df.loc[i]["FamSize"]==0:
        answer[i]*=0.3
    elif df.loc[i]["FamSize"]==1:
        answer[i]*=0.55
    elif df.loc[i]["FamSize"]==2:
        answer[i]*=0.58
    elif df.loc[i]["FamSize"]==3:
        answer[i]*=0.72
    elif df.loc[i]["FamSize"]==4:
        answer[i]*=0.2
    elif df.loc[i]["FamSize"]==5:
        answer[i]*=0.14
    elif df.loc[i]["FamSize"]==6:
        answer[i]*=0.33
    else:
        answer[i]*=0    

In [12]:
for i in range(len(df)):
    if df.loc[i]["Deck"]=="A":
        answer[i]*=0.47
    elif df.loc[i]["Deck"]=="B":
        answer[i]*=0.73
    elif df.loc[i]["Deck"]=="C":
        answer[i]*=0.59
    elif df.loc[i]["Deck"]=="D":
        answer[i]*=0.76
    elif df.loc[i]["Deck"]=="E":
        answer[i]*=0.75
    elif df.loc[i]["Deck"]=="F":
        answer[i]*=0.61
    elif df.loc[i]["Deck"]=="G":
        answer[i]*=0.5
    else:
        answer[i]*=0.3

In [13]:
temp=np.array(answer)
temp=np.around(temp,1)
temp=temp.tolist()
for i in range(len(temp)):
    if temp[i]>=0.1:
        temp[i]=1
    else:
        temp[i]=0

df.insert(6,"Survived",temp)

In [14]:
answer=df[["PassengerId","Survived"]]
answer=pd.DataFrame(answer,columns=["PassengerId","Survived"])
answer.to_csv("answer.csv", sep='\t',index=False)